# Example Python code for rolling dice

In [1]:
import numpy as np
import random

## 1. Rolling a single 6-sided die many times and gathering the results

In [2]:
die_rolls = np.random.randint(1,7, size=100)
die_rolls.view()

array([2, 6, 1, 5, 5, 6, 6, 2, 2, 4, 5, 6, 4, 3, 3, 5, 4, 1, 5, 4, 1, 1,
       4, 1, 2, 2, 4, 5, 2, 2, 3, 3, 6, 6, 2, 1, 3, 3, 1, 3, 2, 5, 6, 2,
       4, 1, 5, 6, 1, 3, 5, 4, 6, 2, 6, 1, 6, 6, 6, 4, 5, 5, 3, 6, 4, 4,
       5, 5, 3, 4, 5, 6, 2, 2, 3, 3, 4, 4, 1, 2, 3, 1, 3, 5, 2, 3, 1, 6,
       5, 6, 2, 3, 3, 1, 4, 2, 2, 4, 4, 3])

In [3]:
number_of_threes = np.count_nonzero(die_rolls==3)
print(number_of_threes)

18


----
We can ask how the dice rolls are distributed.

In [4]:
distributions = {i:np.count_nonzero(die_rolls== i) for i in range(1,7)}
print(distributions)

{1: 14, 2: 18, 3: 18, 4: 17, 5: 16, 6: 17}


----
Or maybe how many odd rolls are followed by even rolls

In [5]:
num_odd_then_even = 0
for i in range(len(die_rolls)-1):
    if die_rolls[i]%2 == 1 and die_rolls[i+1]%2 == 0:
        num_odd_then_even += 1
print(num_odd_then_even)

22


## 2. Rolling Two 6-Sided Dice at Once

In [6]:
two_dice_rolls = np.random.randint(1,7, size=(100,2))
two_dice_rolls[:10]

array([[4, 6],
       [2, 3],
       [2, 4],
       [3, 6],
       [5, 4],
       [1, 2],
       [2, 6],
       [1, 3],
       [3, 2],
       [3, 5]])

----
Summing our two dice

In [7]:
summed_rolls = np.array([sum(row) for row in two_dice_rolls])
summed_rolls.view()

array([10,  5,  6,  9,  9,  3,  8,  4,  5,  8,  7,  5, 11, 10,  7,  7,  5,
       10,  4,  7, 10,  7,  5,  4,  9,  7, 12,  9,  7,  8, 12,  4,  8,  6,
        8,  7,  6,  9, 12,  9,  7,  6,  5,  6,  2,  3,  7,  4, 12, 11, 12,
        9,  7,  2,  3,  7,  8,  9,  5,  9,  6,  9,  8,  6,  7,  4,  6,  8,
        5, 10,  8,  6,  8,  4,  7,  6,  7,  4,  8,  6,  8,  7,  4,  9,  7,
       12,  9,  7,  4,  7, 10,  4,  7,  5,  4,  8,  9,  6,  6,  5])

In [8]:
sum_distributions = {i:np.count_nonzero(summed_rolls== i) for i in range(2,13)}
print(sum_distributions)

{2: 2, 3: 3, 4: 12, 5: 10, 6: 13, 7: 20, 8: 13, 9: 13, 10: 6, 11: 2, 12: 6}


## A Silly Game

This is going to be a very silly two player die rolling game. 

First to score 40 points wins. The first player is determined randomly.

On your turn
* Roll the die, you get the number of points as represented by the roll.
* You may choose to roll again. If you roll high than you last rolled, you double your points. If not, you instead gain no points this turn and your turn is over.
* You may choose to continue to roll again until your turn is over.

I will test two competing strategies.
* Roll once and walk away.
* Roll until you get a 6 or you bust.

---
Strategy 1: The player that just takes what they're given.  I'll call this the `scared_strat`.

In [9]:
def scared_strat():
    return random.randint(1,6)

---
Strategy 2: The player that really likes to double things. I'll call this the `bold_strat`.

In [10]:
def bold_strat():
    roll = random.randint(1,6)
    score = roll
    while roll < 6:
        old_roll = roll
        roll = random.randint(1,6)
        if roll <= old_roll:
            return 0
        else:
            score *= 2
    return score

---
Creating a function that simulates games

In [11]:
def simulate_game(strat_list):
    '''
    Given two strategies for my goofy dice game, determine who wins.
    INPUT::
        - `strat_list`: a list of two functions that each output the points earned by a strategy in a given round
    OUTPUT::
        - the index of the winning strategy from the list
    '''
    points = [0,0] # Players start with 0 points
    who_first = random.randint(0,1) # Determine which player goes first
    who_second = (who_first+1)%2 # Use arithmetic to determine the index of the second player
    while True: # Each round of play is below. The fist person takes their turn and gets points.
        points[who_first] += strat_list[who_first]()
        if points[who_first] >= 40: # If the first player has scored enough, they win.
            return who_first
        else:
            points[who_second] += strat_list[who_second]() # Otherwise the second player goes.
        if points[who_second] >= 40: # If the second player scored enough, then they win.
            return who_second

---
Simulating the game 1000 times to get data on counts. 
There appears to be strong evidence that the bold strategy is the wrong one.

In [12]:
# Simulate 100 times and count how many times each strategy wins
wins = [simulate_game([scared_strat,bold_strat]) for i in range(1000)]
scared_wins = wins.count(0)
print(scared_wins)
bold_wins = wins.count(1)
print(bold_wins)

850
150


In [13]:
def new_strat():
    roll = random.randint(1,6)
    score = roll
    while roll < 4:
        old_roll = roll
        roll = random.randint(1,6)
        if roll <= old_roll:
            return 0
        else:
            score *= 2
    return score

In [14]:
# Simulate 100 times and count how many times each strategy wins
wins = [simulate_game([scared_strat,new_strat]) for i in range(1000)]
scared_wins = wins.count(0)
print(scared_wins)
bold_wins = wins.count(1)
print(bold_wins)

383
617
